In [210]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, GradientBoostingClassifier, GradientBoostingRegressor

In [211]:
df_train = pd.read_csv('./data/train_pre.csv').drop_duplicates()
df_test = pd.read_csv('./data/test_pre.csv').drop_duplicates()
drivers = pd.read_csv('./data/driver_mapping.csv')

drivers.drop(columns=['Unnamed: 0'], inplace=True)

In [212]:
def get_score(pred, actual, margin):
    score = 0
    for i in range(len(pred)):
        if pred[i] == actual[i]:
            score += 1
        elif abs(pred[i] - actual[i]) <= margin:
            score += 1
    return score/len(pred)

#### RF CLASSIFIER

In [213]:
def RF_classifier(X_train, Y_train):
    model = RandomForestClassifier()
    model.fit(X_train, Y_train)

    return model

In [214]:
def get_RF_classifier_Score():
    X_train = df_train
    Y_train = df_train['results_positionOrder']
    model = RF_classifier(X_train.drop(columns = ['results_positionOrder']), Y_train)
    race_nos = []
    predicted_winners = []
    actual_winners = []
    predicted_driver_ids = []
    actual_driver_ids = []
    predicted_winner_name = []
    actual_winner_name = []

    score = 0
    for i in df_test['raceId'].unique():
        X_test = df_test[df_test['raceId'] == i]
        Y_test = df_test[df_test['raceId'] == i]['results_positionOrder']

        prediction = model.predict(X_test.drop(columns = ['results_positionOrder']))        
        actual = Y_test.to_numpy()        
        
        predicted_winner = np.argmin(prediction)
        actual_winner = np.argmin(actual)        
                
        race_nos.append(str(i))
        predicted_winners.append(str(predicted_winner))
        actual_winners.append(str(actual_winner))
        predicted_driver_ids.append(X_test['driverId'].iloc[predicted_winner])
        actual_driver_ids.append(X_test['driverId'].iloc[actual_winner])

        
        predicted_winner_name.append(drivers[np.isclose(drivers['driverId'], X_test['driverId'].iloc[predicted_winner])].driverRef.values[0])
        actual_winner_name.append(drivers[np.isclose(drivers['driverId'], X_test['driverId'].iloc[actual_winner])].driverRef.values[0])

        score += get_score(prediction, actual, 0)

    d = {'Predicted_winner': predicted_winners, 'Predicted Driver' : predicted_winner_name, 'Actual_winner': actual_winners, 'Actual Driver': actual_winner_name}
    return(pd.DataFrame(d, index=race_nos))

In [216]:
print('1951')
get_RF_classifier_Score()

1951


,Predicted_winner,Predicted Driver,Actual_winner,Actual Driver
825,1,fangio,1,fangio
830,1,fangio,4,ascari
831,2,farina,5,ascari
828,1,fangio,1,fangio
832,1,fangio,1,fangio
829,0,fangio,4,trintignant
827,0,fangio,1,farina
826,17,pian,17,pian
